In [2]:
import pandas as pd
from functools import reduce
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder

In [26]:
path = "data_files/"
column_names = ['Calendar Day', 'Stock', 'Moving Average', 'Balance Sheet', 'Cash Flow', 'Income Statement',
                'Shares Outstanding']
metrics = ["BALANCE_SHEET", "CASH_FLOW", "INCOME_STATEMENT", "SHARES_OUTSTANDING"]
stocks = ["AAPL","LLY","NEE"]
daily_metrics = ["DAILY_VOLATILITY", "log_return_MOVING_AVERAGE", "volatility_MOVING_AVERAGE", "volume_MOVING_AVERAGE"]
yearly_metrics = ["BALANCE_SHEET", "CASH_FLOW", "INCOME_STATEMENT"]
years = ['2020', '2021', '2022', '2023', '2024', '2025']

In [27]:
calendar_codes = {"FOMC": 1, "HalfDays": 2, "MonthEnd": 3, "MSCIrebal": 4, "OptionExpiry": 5, "QuarterEnd": 6, "RusselRebal": 7, "TripleWitch": 8}

In [28]:
def combine_datasets_by_stock(stock, daily_metrics):
    stock_dataframes = []
    for metric in daily_metrics:
        df = pd.read_csv(f"{path + stock}_{metric}.csv.gz")
        stock_dataframes.append(df)
    merged_df = reduce(lambda left, right: pd.merge(left, right, on='time_stamp', how='outer'), stock_dataframes)
    merged_df['time_stamp'] = pd.to_datetime(merged_df['time_stamp'])
    merged_df.insert(loc=0, column='Stock', value=stock)
    return merged_df
    

In [29]:
def add_cal_dates(calendar_codes, df):
    df["CALENDAR_DAYS"] = 0
    for day in list(calendar_codes.keys()):
        calendar_df = pd.read_csv(f"{path+day}_CALDATES.csv.gz")
        for i in calendar_df['time_stamp']:
            df.loc[df['time_stamp'] == i, "CALENDAR_DAYS"] = calendar_codes[day]
    return df

In [25]:
quarterly_earnings_df = pd.read_csv(f'data_files/AAPL_EARNINGS.csv.gz')
encoding_dates_df = pd.DataFrame()
quarterly_earnings_df['time_stamp'] = pd.to_datetime(quarterly_earnings_df['time_stamp'])
quarterly_earnings_df = quarterly_earnings_df.sort_values(by='time_stamp', ascending=True)
for earning_release_date in quarterly_earnings_df['time_stamp']:
        df_backward = pd.DataFrame()
        dates_backward = pd.date_range(end=earning_release_date, periods=6, freq="D")
        df_backward["time_stamp"] = dates_backward
        df_backward["QUARTERLY_EARNINGS"] = (earning_release_date - df_backward["time_stamp"]).dt.days
        df_forward = pd.DataFrame()
        dates_forward = pd.date_range(start=earning_release_date, periods=5, freq="D")
        df_forward["time_stamp"] = dates_forward
        df_forward["QUARTERLY_EARNINGS"] = (earning_release_date - df_forward["time_stamp"]).dt.days
        combined = pd.concat([df_forward, df_backward]).sort_values(by="time_stamp", ascending=True).drop_duplicates()
        encoding_dates_df = pd.concat([encoding_dates_df, combined])
print(encoding_dates_df)

   time_stamp  QUARTERLY_EARNINGS
0  2020-01-23                   5
1  2020-01-24                   4
2  2020-01-25                   3
3  2020-01-26                   2
4  2020-01-27                   1
..        ...                 ...
0  2025-10-30                   0
1  2025-10-31                  -1
2  2025-11-01                  -2
3  2025-11-02                  -3
4  2025-11-03                  -4

[240 rows x 2 columns]


In [39]:
#keep track of which quarterly earnings release they are on
#when it passes, go to 0

def earnings_release(df, stock):
    quarterly_earnings_df = pd.read_csv(f'{path+stock}_EARNINGS.csv.gz')
    df["QUARTERLY_EARNINGS"] = 5
    encoding_dates_df = pd.DataFrame()
    quarterly_earnings_df['time_stamp'] = pd.to_datetime(quarterly_earnings_df['time_stamp'])
    quarterly_earnings_df = quarterly_earnings_df.sort_values(by='time_stamp', ascending=True)
    for earning_release_date in quarterly_earnings_df['time_stamp']:
        df_backward = pd.DataFrame()
        dates_backward = pd.date_range(end=earning_release_date, periods=6, freq="D")
        df_backward["time_stamp"] = dates_backward
        df_backward["QUARTERLY_EARNINGS"] = (earning_release_date - df_backward["time_stamp"]).dt.days
        df_forward = pd.DataFrame()
        dates_forward = pd.date_range(start=earning_release_date, periods=5, freq="D")
        df_forward["time_stamp"] = dates_forward
        df_forward["QUARTERLY_EARNINGS"] = (earning_release_date - df_forward["time_stamp"]).dt.days
        combined = pd.concat([df_forward, df_backward]).sort_values(by="time_stamp", ascending=True).drop_duplicates()
        encoding_dates_df = pd.concat([encoding_dates_df, combined])
    mapping = dict(zip(encoding_dates_df["time_stamp"], encoding_dates_df["QUARTERLY_EARNINGS"]))
    print(mapping.keys())
    df.loc[df["time_stamp"].isin(mapping.keys()), "QUARTERLY_EARNINGS"] = df.loc[
        df["time_stamp"].isin(mapping.keys()), "time_stamp"
    ].map(mapping)
    # for earning_release_date in quarterly_earnings_df['time_stamp']:
    #     df_inst = pd.DataFrame()
    #     time_stamp = pd.data_range(end=earning_release_date, periods=5, frequency="D")
        
    #     QUARTERLY_EARNINGS = 


    # df["QUARTERLY_EARNINGS"] = np.nan
    # i = 0
    # for time in list(df['time_stamp']):
    #     if i > 23:
    #         i == 23
    #     x = int((quarterly_earnings_df['time_stamp'].iloc[i] - time).days)
    #     if x == 0:
    #         i += 1
    #         #the next one find the difference between the time and the next quarterly start date
    #     if x > 5:
    #         x = 5
    #     if x < -4:
    #         x = -4
    #     df.loc[df['time_stamp'] == time, "QUARTERLY_EARNINGS"] = x
    return df

In [ ]:
def add_yearly_metrics_revised(df, yearly_metrics, stock, years):
    metric_dataframes = []
    for metric in yearly_metrics:
        print("Processing metric:", metric)
        file_path = f"{path}{stock}_{metric}.csv.gz"
        yearly_metric_df = (
            pd.read_csv(file_path)
            .drop(columns=['Unnamed: 0'], errors='ignore')
            # .dropna(axis=1)
        )
        yearly_metric_df["time_stamp"] = pd.to_datetime(yearly_metric_df["time_stamp"])
        print(yearly_metric_df)

    #     year_dfs = []
    #     all_dates_df = pd.DataFrame({"time_stamp": df["time_stamp"]})

    #     for year in years:
    #         mask = all_dates_df["time_stamp"].dt.year == int(year)
    #         year_specific_df = all_dates_df.loc[mask].copy()
    #         year_specific_data = yearly_metric_df[
    #             yearly_metric_df["time_stamp"].dt.year == int(year)
    #         ].drop(columns=["time_stamp"], errors="ignore")

    #         if not year_specific_data.empty:
    #             for measurement in year_specific_data.columns:
    #                 value = year_specific_data[measurement].values[0]
    #                 year_specific_df.loc[:, measurement] = value
    #         year_dfs.append(year_specific_df)
    #     metric_df = pd.concat(year_dfs, ignore_index=False)
    #     metric_dataframes.append(metric_df)

    # all_metrics_df = reduce(
    #     lambda left, right: pd.merge(left, right, on="time_stamp", how="outer"),
    #     metric_dataframes
    # )
    # result_df = pd.merge(df, all_metrics_df, on="time_stamp", how="inner")
    return result_df

In [ ]:
def add_shares_outstanding(stock, df):
    shares_df = pd.read_csv(f"{path+stock}_SHARES_OUTSTANDING.csv.gz")
    df["Month-Year"] = str(df["time_stamp"].dt.month + "-" df["time_stamp"].dt.year)
    month_year_df = pd.DataFrame({"Month-Year": df["Month-Year"]})
    
    

In [63]:
stock_df = []
for stock in stocks:
    df = combine_datasets_by_stock(stock, daily_metrics)
    df = add_cal_dates(calendar_codes, df)
    df = earnings_release(df, stock)
    df = add_yearly_metrics_revised(df, yearly_metrics, stock, years)
    
#     stock_df.append(df)
# final_dataset = pd.concat(stock_df, axis=0, ignore_index=False).dropna()
# le = LabelEncoder()
# final_dataset["Stock"] = le.fit_transform(final_dataset["Stock"])
# final_dataset['time_stamp'] = final_dataset['time_stamp'].astype('int64') // 10**9
# final_dataset = final_dataset.drop(columns=['BALANCE_SHEET: reportedCurrency', 'INCOME_STATEMENT: reportedCurrency', 'CASH_FLOW: reportedCurrency'])
# final_dataset
# final_dataset.to_csv("final_dataset4.csv")


dict_keys([Timestamp('2020-01-23 00:00:00'), Timestamp('2020-01-24 00:00:00'), Timestamp('2020-01-25 00:00:00'), Timestamp('2020-01-26 00:00:00'), Timestamp('2020-01-27 00:00:00'), Timestamp('2020-01-28 00:00:00'), Timestamp('2020-01-29 00:00:00'), Timestamp('2020-01-30 00:00:00'), Timestamp('2020-01-31 00:00:00'), Timestamp('2020-02-01 00:00:00'), Timestamp('2020-04-25 00:00:00'), Timestamp('2020-04-26 00:00:00'), Timestamp('2020-04-27 00:00:00'), Timestamp('2020-04-28 00:00:00'), Timestamp('2020-04-29 00:00:00'), Timestamp('2020-04-30 00:00:00'), Timestamp('2020-05-01 00:00:00'), Timestamp('2020-05-02 00:00:00'), Timestamp('2020-05-03 00:00:00'), Timestamp('2020-05-04 00:00:00'), Timestamp('2020-07-25 00:00:00'), Timestamp('2020-07-26 00:00:00'), Timestamp('2020-07-27 00:00:00'), Timestamp('2020-07-28 00:00:00'), Timestamp('2020-07-29 00:00:00'), Timestamp('2020-07-30 00:00:00'), Timestamp('2020-07-31 00:00:00'), Timestamp('2020-08-01 00:00:00'), Timestamp('2020-08-02 00:00:00'), Tim

In [64]:
df

,Stock,time_stamp,volatility,MA_1_log_return,MA_5_log_return,MA_22_log_return,MA_44_log_return,MA_1_volatility,MA_5_volatility,MA_22_volatility,...,INCOME_STATEMENT: costOfRevenue,INCOME_STATEMENT: costofGoodsAndServicesSold,INCOME_STATEMENT: operatingIncome,INCOME_STATEMENT: operatingExpenses,INCOME_STATEMENT: depreciationAndAmortization,INCOME_STATEMENT: incomeBeforeTax,INCOME_STATEMENT: incomeTaxExpense,INCOME_STATEMENT: ebit,INCOME_STATEMENT: ebitda,INCOME_STATEMENT: netIncome
0,NEE,2020-01-02,0.000380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.290000e+09,7.290000e+09,5.116000e+09,5.591000e+09,4.315000e+09,2.413000e+09,44000000.0,4.363000e+09,8.678000e+09,2.919000e+09
1,NEE,2020-01-03,0.000350,-0.015426,NaN,NaN,NaN,0.000380,NaN,NaN,...,7.290000e+09,7.290000e+09,5.116000e+09,5.591000e+09,4.315000e+09,2.413000e+09,44000000.0,4.363000e+09,8.678000e+09,2.919000e+09
2,NEE,2020-01-06,0.000321,0.002780,NaN,NaN,NaN,0.000350,NaN,NaN,...,7.290000e+09,7.290000e+09,5.116000e+09,5.591000e+09,4.315000e+09,2.413000e+09,44000000.0,4.363000e+09,8.678000e+09,2.919000e+09
3,NEE,2020-01-07,0.000316,-0.006470,NaN,NaN,NaN,0.000321,NaN,NaN,...,7.290000e+09,7.290000e+09,5.116000e+09,5.591000e+09,4.315000e+09,2.413000e+09,44000000.0,4.363000e+09,8.678000e+09,2.919000e+09
4,NEE,2020-01-08,0.000303,0.005582,NaN,NaN,NaN,0.000316,NaN,NaN,...,7.290000e+09,7.290000e+09,5.116000e+09,5.591000e+09,4.315000e+09,2.413000e+09,44000000.0,4.363000e+09,8.678000e+09,2.919000e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,NEE,2025-09-24,0.000531,0.003252,0.007066,-0.001803,-0.002873,0.000545,0.000596,0.000626,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1440,NEE,2025-09-25,0.000777,0.017941,0.009702,-0.000715,-0.000862,0.000531,0.000590,0.000615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1441,NEE,2025-09-26,0.000621,0.009997,0.009534,0.000132,-0.000424,0.000777,0.000613,0.000625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1442,NEE,2025-09-29,0.000592,0.014341,0.012690,0.001169,0.000095,0.000621,0.000615,0.000633,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
